In [2]:
import pandas as pd
import requests
import json
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import calendar
from datetime import date
import holidays
import statsmodels.api as sm

In [3]:
import time

In [4]:
import xgboost as xgb

In [5]:
from sklearn import preprocessing 

In [6]:
train = pd.read_csv('ashrae-energy-prediction/train.csv')
building = pd.read_csv('ashrae-energy-prediction/building_metadata.csv')
weather_data_tr=pd.read_csv('ashrae-energy-prediction/weather_train.csv')
test = pd.read_csv('ashrae-energy-prediction/test.csv')
weather_data_te=pd.read_csv('ashrae-energy-prediction/weather_test.csv')
submission=pd.read_csv('ashrae-energy-prediction/sample_submission.csv')

In [7]:
submission=pd.read_csv('ashrae-energy-prediction/sample_submission.csv')

In [8]:
from gimme_data import all_the_data

In [9]:
df=all_the_data(train, building, weather_data_tr)

In [1]:
df.head()

NameError: name 'df' is not defined

In [7]:
df_no_dummies=all_the_data(train, building, weather_data_tr,dummies_=False)

In [8]:
df_no_dummies.head()

,building_id,timestamp,meter,meter_reading,site_id,month,hour,weekday,holiday,0_electricity,1_chilledwater,2_steam,3_hotwater,air_temperature,dew_temperature,wind_speed
0,0,2016-01-01,0,0.0,0,1,0,4,True,0.0,0.0,0.0,0.0,25.0,20.0,0.0
1,1,2016-01-01,0,0.0,0,1,0,4,True,0.0,0.0,0.0,0.0,25.0,20.0,0.0
2,2,2016-01-01,0,0.0,0,1,0,4,True,0.0,0.0,0.0,0.0,25.0,20.0,0.0
3,3,2016-01-01,0,0.0,0,1,0,4,True,0.0,0.0,0.0,0.0,25.0,20.0,0.0
4,4,2016-01-01,0,0.0,0,1,0,4,True,0.0,0.0,0.0,0.0,25.0,20.0,0.0


## XGBoost
train- 2016/01/01-2016/12/31

test-  2017/01/01-2018/05/09 7:00

In [14]:
df0.head()

NameError: name 'df0' is not defined

In [15]:
#Add Building type to df
# df=df.join(pd.get_dummies(building['primary_use']),how='left',on='building_id')
# #Add site_id to list
# df=df.join(pd.get_dummies(building['site_id'],prefix='site_'),how='left',on='building_id')
# #Add sqr footage to df
# min_max_scaler = preprocessing.MinMaxScaler()
# x=min_max_scaler.fit_transform(pd.DataFrame(building['square_feet']))
# building['std_sqr_ft']=x
# df=df.join(building['std_sqr_ft'],how='left',on='building_id')
# dfs=[]
# for i in range(4):
#     interior_dfs=[]
#     df_ex=df.loc[df[i]==1]
#     for i in range(16):
        
#         interior_dfs.append()
#     dfs.append(df.loc[df[0]==1])
df0=df.loc[df[0]==1]
df1=df.loc[df[1]==1]
df2=df.loc[df[2]==1]
df3=df.loc[df[3]==1]


#check that all variable are there
full=0
full+=df0.shape[0]
full+=df1.shape[0]
full+=df2.shape[0]
full+=df3.shape[0]
print(full)
print(df.shape[0])

20216100
20216100


In [97]:
df=df.join(pd.get_dummies(building['site_id'],prefix='site_'),how='left',on='building_id')

In [51]:
building=pd.concat([building,pd.get_dummies(building['primary_use'])],axis=1)

In [106]:
building['site_id'].value_counts().shape

(16,)

In [82]:
building.columns

Index(['site_id', 'building_id', 'primary_use', 'square_feet', 'year_built',
       'floor_count', 'Education', 'Entertainment/public assembly',
       'Food sales and service', 'Healthcare', 'Lodging/residential',
       'Manufacturing/industrial', 'Office', 'Other', 'Parking',
       'Public services', 'Religious worship', 'Retail', 'Services',
       'Technology/science', 'Utility', 'Warehouse/storage', 'log_sqr_ft',
       'std_sqr_ft'],
      dtype='object')

In [102]:
df0.shape

(12060910, 49)

In [107]:
df1.columns

Index([                  'building_id',                     'timestamp',
                               'meter',                 'meter_reading',
                             'site_id',                         'month',
                                'hour',                       'weekday',
                             'holiday',                               0,
                                     1,                               2,
                                     3,               'air_temperature',
                     'dew_temperature',                    'wind_speed',
                           'Education', 'Entertainment/public assembly',
              'Food sales and service',                    'Healthcare',
                 'Lodging/residential',      'Manufacturing/industrial',
                              'Office',                         'Other',
                             'Parking',               'Public services',
                   'Religious worship',            

In [95]:
df0['meter_reading']

0             0.000
1             0.000
2             0.000
3             0.000
4             0.000
             ...   
20216095      8.750
20216096      4.825
20216097      0.000
20216098    159.575
20216099      2.850
Name: meter_reading, Length: 12060910, dtype: float64

In [94]:
train.tail()

,building_id,meter,timestamp,meter_reading
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575
20216099,1448,0,2016-12-31 23:00:00,2.850


In [ ]:
train_cols=['month', 'hour', 'weekday','holiday', 'air_temperature',
                     'dew_temperature',                    'wind_speed',
                           'Education', 'Entertainment/public assembly',
              'Food sales and service',                    'Healthcare',
                 'Lodging/residential',      'Manufacturing/industrial',
                              'Office',                         'Other',
                             'Parking',               'Public services',
                   'Religious worship',                        'Retail',
                            'Services',            'Technology/science',
                             'Utility',             'Warehouse/storage',
                             'site__0',                       'site__1',
                             'site__2',                       'site__3',
                             'site__4',                       'site__5',
                             'site__6',                       'site__7',
                             'site__8',                       'site__9',
                            'site__10',                      'site__11',
                            'site__12',                      'site__13',
                            'site__14',                      'site__15',
                          'std_sqr_ft']

In [11]:
y0=df0['meter_reading']
X0=df0[train_cols]

NameError: name 'df0' is not defined

In [ ]:
X0.shape

In [114]:
clf = xgb.XGBClassifier()

In [ ]:
start=time.time()
clf.fit(X0, y0)
end=time.time()
print(end-start)

[21:51:25] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [ ]:
def add_building_cols(dfx,dfy,building)
    dfx=all_the_data(dfy, building, weather_data_te)
    #Add Building type to df
    dfx=dfx.join(pd.get_dummies(building['primary_use']),how='left',on='building_id')
    #Add site_id to list
    dfx=dfx.join(pd.get_dummies(building['site_id'],prefix='site_'),how='left',on='building_id')
    #Add sqr footage to df
    min_max_scaler = preprocessing.MinMaxScaler()
    x=min_max_scaler.fit_transform(pd.DataFrame(building['square_feet']))
    building['std_sqr_ft']=x
    dfx=dfx.join(building['std_sqr_ft'],how='left',on='building_id')
    dfs=[]
    for i in range(4):
        interior_dfs=[]
        df_ex=dfx.loc[dfx[i]==1]
        for i in range(16):

            interior_dfs.append()
        dfs.append(dfx.loc[dfx[0]==1])
    df0=dfx.loc[df[0]==1]
    df1=dfx.loc[df[1]==1]
    df2=dfx.loc[df[2]==1]
    df3=dfx.loc[df[3]==1]


    #check that all variable are there
    full=0
    full+=df0.shape[0]
    full+=df1.shape[0]
    full+=df2.shape[0]
    full+=df3.shape[0]
    print(full)
    print(df.shape[0])